# Probabilistic Forecasting

This notebook demonstrates the usage of autoregressive deep neural prediction models on synethic time-series sales data. The data generator produces a rich synthetic dataset with many-to-many relationships such as product-location pairs. 

The demo code requires the following requirements to be met: 

```
gluonts>=0.8.1
pandas>=1.3
torch>=1.9
numpy>=1.21.4
```

### Example Usage

To begin, we import a few libraries that will help in generating some synthetic sales data. 

In [ ]:
import random
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

First, we'll declare an Enum object called `SalesDataModel` to represent the different types of sales data trends we can generate: 

In [ ]:
from enum import Enum

class SalesDataModel(Enum):
    RANDOM = 0
    SINUSOID = 1
    SPARSE = 2

Next, we define the `SalesDataGenerator` class that takes a couple of arguments that define the characteristics of the data to be generated:

In [ ]:
from typing import List, Mapping

class SalesDataGenerator:
    def __init__(
        self,
        start_date: datetime,
        num_days: int,
        item_to_location: Mapping[str, List[str]],
        item_to_class: Mapping[str, List[str]],
        min_sales_per_day: int = 0,
        max_sales_per_day: int = 1000,
    ):
        self.start_date = start_date
        self.num_days = num_days
        self.item_to_location = item_to_location
        self.item_to_class = item_to_class
        self.min_sales_per_day = min_sales_per_day
        self.max_sales_per_day = max_sales_per_day

    def synthesize(self, data_model: SalesDataModel = SalesDataModel.RANDOM):
        df = pd.DataFrame(columns=["Date", "Item", "Location", "Sales"])
        for item, locations in self.item_to_location.items():
            for location in locations:
                if data_model == SalesDataModel.RANDOM:
                    sales = np.random.randint(
                        self.min_sales_per_day, self.max_sales_per_day, self.num_days
                    )
                elif data_model == SalesDataModel.SINUSOID:
                    alpha = 0.9
                    gain = alpha * (self.max_sales_per_day - self.min_sales_per_day)
                    periods = np.random.randint(1, 6, 1)
                    nn = np.linspace(0, int(periods), self.num_days)
                    base_sales = (
                        gain * np.abs(np.cos(np.pi * nn))
                        + alpha * self.min_sales_per_day
                    )
                    noise = np.random.randint(
                        (1 - alpha) * self.min_sales_per_day,
                        (1 - alpha) * self.max_sales_per_day,
                        self.num_days,
                    )
                    sales = base_sales + noise
                    sales = sales.astype(int).tolist()
                elif data_model == SalesDataModel.SPARSE:
                    thresh = 0.1
                    sales = [
                        0
                        if np.random.rand() > thresh
                        else np.random.randint(
                            self.min_sales_per_day, self.max_sales_per_day, 1
                        )
                        for _ in range(self.num_days)
                    ]
                _df = pd.DataFrame(
                    {
                        "Date": [
                            self.start_date + timedelta(days=diff)
                            for diff in range(self.num_days)
                        ],
                        "Item": [item] * self.num_days,
                        "Location": [location] * self.num_days,
                        "Sales": sales,
                        "Item Class": [self.item_to_class[item]] * self.num_days,
                    }
                )
                df = pd.concat([df, _df], ignore_index=True)
        return df

For example, let's set the parameters to generate sales data over 5 years (`num_days`) backwards from today. We'll include 20 items, with each item being sold at 5 locations. Each item belongs to a item class randomly selected from 3 classes (this will be a static covariate when we get to the modeling section). 

In [ ]:
num_days = 365 * 5
items = [f"Item {idx}" for idx in range(20)]
locations = [f"Location {idx}" for idx in range(5)]
item_to_class = {
    item: random.choice(["Item Class A", "Item Class B", "Item Class C"])
    for item in items
}

Next, we generate the data. For this example, we'll generate sinusoidal data (which also incorporates noise). Note that the `min_sales_per_day` argument is negative, meaning that the net sales for the day could be items returned. 

In [ ]:
generator = SalesDataGenerator(
    start_date=datetime.today().date() - timedelta(days=num_days),
    num_days=num_days,
    item_to_location={item: locations for item in items},
    item_to_class=item_to_class,
    max_sales_per_day=1000,
    min_sales_per_day=-100,
)
data = generator.synthesize(data_model=SalesDataModel.SINUSOID)

The object generated is a pandas dataframe, so we can peak at the data as follows: 

In [ ]:
data.head()

With the dataset defined, we next need to preprocess it compatable with the model training object. To do this, we create a list of dictionaries with specific keys. Note that we are scaling the data in this preprocessing step to be compatable with the statistical structure of the model we define later. 

In [ ]:
start = data.Date.min()
end = data.Date.max()
ensemble = []
dates = list(pd.date_range(start, end))
for item in set(data.Item): 
    for location in set(data.Location):
        sales = data[(data.Item == item) & (data.Location == location)].Sales.values
        if len(sales): 
            scale = max(abs(sales))
            ensemble.append(
                {
                    "class": item_to_class[item],
                    "item": item,
                    "sales": sales,
                    "start": start,
                    "end": end,
                    "total": sum(sales),
                    "scale": scale,
                    "location": location,
                    "sales_scaled": sales / scale,
                }
            )
ensemble = sorted(ensemble, key=lambda k: k["total"], reverse=True)

The probabalistic neural model we are using requires specific objects to be created to capture the (1) time series data and (2) covariates. The following code snippet produces these: 

In [ ]:
from gluonts.dataset.common import load_datasets, ListDataset
from gluonts.dataset.field_names import FieldName

quantity_length = len(ensemble[0]["sales"])
prediction_length = 30
context_length = 6 * prediction_length

static_cats = [(e["class"].strip(), e["item"].strip(), e["location"].strip()) for e in ensemble]

item_to_index = {item: idx for idx, item in enumerate(set([sc[0] for sc in static_cats]))}
item_class_to_index = {item: idx for idx, item in enumerate(set([sc[1] for sc in static_cats]))}
location_to_index = {loc: idx for idx, loc in enumerate(set([sc[2] for sc in static_cats]))}

test_target_values = np.array([np.array(e["sales_scaled"]) for e in ensemble])
train_target_values = np.array([np.array(e["sales_scaled"][:-prediction_length]) for e in ensemble])

Next, we build the training and testing data splits:

In [ ]:
train_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start,
        FieldName.FEAT_STATIC_CAT: [
            item_to_index[sc_item], 
            item_class_to_index[sc_item_class], 
            location_to_index[sc_location]
        ],
    }
    for target, (sc_item, sc_item_class, sc_location) in zip(train_target_values, static_cats)
], freq="D")

test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start,
        FieldName.FEAT_STATIC_CAT: [
            item_to_index[sc_item], 
            item_class_to_index[sc_item_class], 
            location_to_index[sc_location]
        ],
    }
    for target, (sc_item, sc_item_class, sc_location) in zip(test_target_values, static_cats)
], freq="D")

The model architecture is defined below:

In [ ]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.distribution.gaussian import GaussianOutput
from gluonts.mx.trainer import Trainer

estimator = DeepAREstimator(
    prediction_length=prediction_length,
    context_length=context_length,
    freq="D",
    num_cells = 2,
    distr_output = GaussianOutput(),
    use_feat_dynamic_real=False,
    use_feat_static_cat=True,
    cardinality=[len(item_to_index), len(item_class_to_index), len(location_to_index)],
    embedding_dimension=[32,64, 32],
    trainer=Trainer(
        learning_rate=1e-2,
        epochs=20,
        num_batches_per_epoch=10,
        clip_gradient=1.0
    )
)

Commence model training!

In [ ]:
predictor = estimator.train(train_ds)

With the model trained, we now look to evaluate the performance on the test dataset:

In [ ]:
from gluonts.evaluation.backtest import make_evaluation_predictions

forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=predictor, num_samples=250)
forecasts = [f for f in forecast_it]
tss = list(ts_it)

from gluonts.evaluation import make_evaluation_predictions, Evaluator

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
metrics, _ = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds))
metrics = pd.DataFrame.from_records(metrics, index=["DeepAR"]).transpose()
metrics

To help visualize the results, we next plot a few of the probabalistic predictions:

In [ ]:
import copy
import matplotlib.pyplot as plt
from itertools import islice
plt.figure(figsize=(20, 15))
plt.rcParams.update({'font.size': 15})

for idx, (f, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ensemble[idx]['scale'] * ts[-15 * prediction_length:], label="target")
    f2 = copy.deepcopy(f)
    f2.samples *= ensemble[idx]['scale']
    f2.plot()
    plt.xticks(rotation=60)
    plt.title(f"{ensemble[idx]['item']}")

plt.gcf().tight_layout()
plt.legend()
plt.show()